In [1]:
import pandas as pd
import numpy as np
import re
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings("ignore")

In [1]:
DATA_PATH = '../data/data_set_0926/'
FEAT_PATH = './features/'

In [2]:
tic = time.time()
invite_info = pd.read_table(os.path.join(DATA_PATH, 'invite_info_0926.txt'), header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']
    
invite_info_evaluate_A = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_1_0926.txt'), header=None)
invite_info_evaluate_A.columns =  ['问题ID','用户ID','邀请创建时间']

invite_info_evaluate_B = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_2_0926.txt'), header=None)
invite_info_evaluate_B.columns =  ['问题ID','用户ID','邀请创建时间']

oversample = True

if oversample:
    data = pd.concat([invite_info, invite_info_evaluate_B, invite_info_evaluate_A], axis=0)
else:
    data = pd.concat([invite_info, invite_info_evaluate_B], axis=0)
    
data = data.merge(question_info, on='问题ID', how='left')
print("Used time: %d s" % (time.time()-tic))

Used time: 15 s


In [3]:
tic = time.time()
tmp = data['邀请创建时间'].apply(lambda x : x.split('-'))
data['邀请创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
data['邀请创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))
data['邀请创建时间_DH'] = data['邀请创建时间_D']*24 + data['邀请创建时间_H']
    
tmp = data[['用户ID', '邀请创建时间_DH']].groupby(['用户ID', '邀请创建时间_DH']).size().reset_index()
tmp.columns = ['用户ID', '邀请创建时间_DH', '用户ID_expo']
tmp = tmp.sort_values(by=['用户ID','邀请创建时间_DH'])
tmp['用户ID_last_expo'] = tmp.groupby('用户ID')['用户ID_expo'].cumsum() - tmp['用户ID_expo']
tmp = tmp.sort_values(by=['用户ID','邀请创建时间_DH'], ascending=False)
tmp['用户ID_next_expo'] = tmp.groupby('用户ID')['用户ID_expo'].cumsum() - tmp['用户ID_expo']
data = data.merge(tmp, 'left', ['用户ID','邀请创建时间_DH'])    
    
tmp = data[['问题ID', '邀请创建时间_DH']].groupby(['问题ID', '邀请创建时间_DH']).size().reset_index()
tmp.columns = ['问题ID', '邀请创建时间_DH', '问题ID_expo']
tmp = tmp.sort_values(by=['问题ID','邀请创建时间_DH'])
tmp['问题ID_last_expo'] = tmp.groupby('问题ID')['问题ID_expo'].cumsum() - tmp['问题ID_expo']
tmp = tmp.sort_values(by=['问题ID','邀请创建时间_DH'], ascending=False)
tmp['问题ID_next_expo'] = tmp.groupby('问题ID')['问题ID_expo'].cumsum() - tmp['问题ID_expo']
data = data.merge(tmp, 'left', ['问题ID','邀请创建时间_DH'])
    
tmp = data[['用户ID_last_expo','用户ID_next_expo','问题ID_last_expo','问题ID_next_expo']]
print("Used time: %d s" % (time.time()-tic))

Used time: 145 s


In [4]:
tmp.to_pickle(os.path.join(FEAT_PATH, 'last_next_expo_feat.pickle'))
print("Feature Saved, shape:",tmp.shape)

Feature Saved, shape: (11772563, 4)
